In [4]:
from pyspark.sql import SparkSession

# Create or retrieve the Spark session
spark = SparkSession.builder \
    .appName("CS5052 Data Analysis") \
    .getOrCreate()

# Set the log level to WARN to reduce log output
spark.sparkContext.setLogLevel("WARN")

# Print Spark session version to verify it's running
print(f"Spark session started with version: {spark.version}")

# Read the dataset
raw_data = spark.read.csv("../data/raw/Absence_3term201819_nat_reg_la_sch.csv", header=True)

# Show initial data sample
raw_data.show(5)

Spark session started with version: 3.5.5


25/03/19 11:23:56 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------------+--------------+-----------+----------------+------------+------------+-----------+-----------+-----------+-----------+-------+-----+-------+----+--------------------+------------+-----------------+-----------+-----------+----------+-------------+------------+---------------+-----------------+--------------------+-----------------------+-------------------------+----------------------+------------------------------+-------------------------+------------------------+---------------------------+-----------------------------+--------------------------------+-----------------------------------+-------------------------------------+-----------------+----------------------+-------------------+---------------+-------------------+-----------------+---------------------+------------------+---------------+----------------------+-------------------+----------------+-----------------+-----------------+------------------------+-------------------------+
|time_identifier|year_breakdo

In [45]:
# Read the dataset and filter for Regional level
regional_data = raw_data.filter((raw_data.geographic_level == 'Local authority') & 
                               (raw_data.time_period == '201819')).orderBy("la_name")

# Show the filtered results
regional_data.show()

+---------------+--------------+-----------+----------------+------------+------------+-----------+--------------------+-----------+-----------+--------------------+-----+-------+----+--------------------+------------+-----------------+-----------+-----------+----------+-------------+------------+---------------+-----------------+--------------------+-----------------------+-------------------------+----------------------+------------------------------+-------------------------+------------------------+---------------------------+-----------------------------+--------------------------------+-----------------------------------+-------------------------------------+-----------------+----------------------+-------------------+---------------+-------------------+-----------------+---------------------+------------------+---------------+----------------------+-------------------+----------------+-----------------+-----------------+------------------------+-------------------------+
|time_i

In [50]:
# Read the dataset and filter for Regional level
regional_data = raw_data.filter((raw_data.geographic_level == 'Local authority') & 
                               (raw_data.time_period == '201819') & 
                               (raw_data.la_name == "Barnsley")).agg({'enrolments': 'sum'})
regional_data.show()

regional_data = raw_data.filter((raw_data.geographic_level == 'Local authority') & 
                                (raw_data.school_type == 'Total') &
                               (raw_data.time_period == '201819') & 
                               (raw_data.la_name == "Barnsley")).agg({'enrolments': 'sum'})

regional_data.show()

regional_data = raw_data.filter((raw_data.geographic_level == 'Local authority') & 
                                (raw_data.school_type != 'Total') &
                               (raw_data.time_period == '201819') & 
                               (raw_data.la_name == "Barnsley")).agg({'enrolments': 'sum'})

regional_data.show()


+---------------+
|sum(enrolments)|
+---------------+
|        60090.0|
+---------------+

+---------------+
|sum(enrolments)|
+---------------+
|        30045.0|
+---------------+

+---------------+
|sum(enrolments)|
+---------------+
|        30045.0|
+---------------+

